In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%capture
!unzip /content/gdrive/MyDrive/test-archive.zip -d /content/testing-dataset

In [ ]:
!mv "/content/testing-dataset/development_test_data/gallery" "/content/"
!mv "/content/testing-dataset/development_test_data/queries" "/content/"

In [ ]:
!mkdir "/content/original_datasets"
!mv "/content/testing-dataset/development_test_data/gallery.csv" "/content/original_datasets/"
!mv "/content/testing-dataset/development_test_data/queries.csv" "/content/original_datasets/"

In [ ]:
!rm -rf "/content/testing-dataset"

In [ ]:
%%capture
!unzip '/content/gdrive/MyDrive/gallery_tensors_bt_0.4_tt_0.25.zip' -d '/'
!unzip '/content/gdrive/MyDrive/query_tensors_bt_0.4_tt_0.25.zip' -d '/'
!unzip '/content/gdrive/MyDrive/gallery_tensors_detection_bbox_crop_bt_0.4_tt_0.25.zip' -d '/'
!unzip '/content/gdrive/MyDrive/query_tensors_detection_bbox_crop_bt_0.4_tt_0.25.zip' -d '/'
!unzip '/content/gdrive/MyDrive/gallery_tensors_blip_bt_0.4_tt_0.25.zip' -d '/'
!unzip '/content/gdrive/MyDrive/query_tensors_blip_bt_0.4_tt_0.25.zip' -d '/'
!unzip '/content/gdrive/MyDrive/gallery_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25.zip' -d '/'
!unzip '/content/gdrive/MyDrive/query_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25.zip' -d '/'

In [ ]:
!rm -rf content

In [ ]:
import os
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torchvision.io import read_image
from torch.utils.data import Dataset
import pandas as pd
import torchvision.transforms as T
import torchvision.transforms.functional as F
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# gallery_tensor_dataset_path = '/content/gallery_tensors_bt_0.4_tt_0.25/gallery_tensors_bt_0.4_tt_0.25.csv'
# query_tensor_dataset_path = '/content/query_tensors_bt_0.4_tt_0.25/query_tensors_bt_0.4_tt_0.25.csv'


# gallery_tensor_dataset_path = '/content/gallery_tensors_detection_bbox_crop_bt_0.4_tt_0.25/gallery_tensors_detection_bbox_crop_bt_0.4_tt_0.25.csv'
# query_tensor_dataset_path = '/content/query_tensors_detection_bbox_crop_bt_0.4_tt_0.25/query_tensors_detection_bbox_crop_bt_0.4_tt_0.25.csv'


# gallery_tensor_dataset_path = '/content/gallery_tensors_blip_bt_0.4_tt_0.25/gallery_tensors_blip_bt_0.4_tt_0.25.csv'
# query_tensor_dataset_path = '/content/query_tensors_blip_bt_0.4_tt_0.25/query_tensors_blip_bt_0.4_tt_0.25.csv'


gallery_tensor_dataset_path = '/content/gallery_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25/gallery_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25.csv'
query_tensor_dataset_path = '/content/query_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25/query_tensors_blip_detection_bbox_crop_bt_0.4_tt_0.25.csv'

In [ ]:
gallery_tensors_df = pd.read_csv(gallery_tensor_dataset_path, low_memory=False)
query_tensors_df = pd.read_csv(query_tensor_dataset_path, low_memory=False)
print('Gallery Tensor Dataset\n==========================================')
display(gallery_tensors_df.head())
print('\nQuery Tensor Dataset\n==========================================')
display(query_tensors_df.head())

Gallery Tensor Dataset


,index,image_tensor_path,masked_image_tensor_path,inverse_masked_image_tensor_path,label,image_path,masked_similarity,inverse_masked_similarity
0,0,gallery_tensors_bt_0.4_tt_0.25/image_0_2.pt,gallery_tensors_bt_0.4_tt_0.25/masked_image_0_...,gallery_tensors_bt_0.4_tt_0.25/inverse_masked_...,5019,gallery/ambitious-tough-teal-from-asgard.jpg,22.579002,2.187292
1,1,gallery_tensors_bt_0.4_tt_0.25/image_1_2.pt,gallery_tensors_bt_0.4_tt_0.25/masked_image_1_...,gallery_tensors_bt_0.4_tt_0.25/inverse_masked_...,5008,gallery/fine-shrewd-oarfish-of-genius.jpg,5.359891,18.161760
2,2,gallery_tensors_bt_0.4_tt_0.25/image_2_2.pt,gallery_tensors_bt_0.4_tt_0.25/masked_image_2_...,gallery_tensors_bt_0.4_tt_0.25/inverse_masked_...,9,gallery/bold-nickel-gecko-of-reputation.jpg,1.140360,14.966935
3,3,gallery_tensors_bt_0.4_tt_0.25/image_3_2.pt,gallery_tensors_bt_0.4_tt_0.25/masked_image_3_...,gallery_tensors_bt_0.4_tt_0.25/inverse_masked_...,3589,gallery/marvellous-uber-boobook-of-lightning.jpg,3.918887,10.602951
4,4,gallery_tensors_bt_0.4_tt_0.25/image_4_2.pt,gallery_tensors_bt_0.4_tt_0.25/masked_image_4_...,gallery_tensors_bt_0.4_tt_0.25/inverse_masked_...,3656,gallery/bouncy-economic-agama-of-honeydew.jpg,2.593714,16.728163



Query Tensor Dataset


,index,image_tensor_path,masked_image_tensor_path,inverse_masked_image_tensor_path,label,image_path,masked_similarity,inverse_masked_similarity
0,0,query_tensors_bt_0.4_tt_0.25/image_0_2.pt,query_tensors_bt_0.4_tt_0.25/masked_image_0_2.pt,query_tensors_bt_0.4_tt_0.25/inverse_masked_im...,5013,queries/magnetic-powerful-platypus-of-hail.jpeg,18.659145,3.107853
1,1,query_tensors_bt_0.4_tt_0.25/image_1_2.pt,query_tensors_bt_0.4_tt_0.25/masked_image_1_2.pt,query_tensors_bt_0.4_tt_0.25/inverse_masked_im...,3550,queries/free-keen-mole-of-cookies.jpeg,21.546976,3.820572
2,2,query_tensors_bt_0.4_tt_0.25/image_2_2.pt,query_tensors_bt_0.4_tt_0.25/masked_image_2_2.pt,query_tensors_bt_0.4_tt_0.25/inverse_masked_im...,71,queries/hilarious-precious-parakeet-of-adverti...,6.881780,20.275496
3,3,query_tensors_bt_0.4_tt_0.25/image_3_2.pt,query_tensors_bt_0.4_tt_0.25/masked_image_3_2.pt,query_tensors_bt_0.4_tt_0.25/inverse_masked_im...,3566,queries/optimal-uptight-ringtail-of-cleaning.jpeg,21.791758,6.283848
4,4,query_tensors_bt_0.4_tt_0.25/image_4_2.pt,query_tensors_bt_0.4_tt_0.25/masked_image_4_2.pt,query_tensors_bt_0.4_tt_0.25/inverse_masked_im...,5329,queries/nonchalant-impala-of-fabulous-artistry...,12.197595,19.285637


In [ ]:
class TensorDataset(Dataset):
    def __init__(self, annotations_file, tensor_dir, transform=None, target_transform=None):
        self.tensor_labels = pd.read_csv(annotations_file)
        self.tensor_dir = tensor_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.tensor_labels)

    def __getitem__(self, idx):
        image_tensor_path = os.path.join(self.tensor_dir, self.tensor_labels.iloc[idx, 1])
        masked_tensor_path = os.path.join(self.tensor_dir, self.tensor_labels.iloc[idx, 2])
        inverse_masked_tensor_path = os.path.join(self.tensor_dir, self.tensor_labels.iloc[idx, 3])

        image = torch.load(image_tensor_path)
        if masked_tensor_path != 'None' and inverse_masked_tensor_path != None:
          masked = torch.load(masked_tensor_path)
          inverse_masked = torch.load(inverse_masked_tensor_path)
        else:
          masked = torch.empty(0)
          inverse_masked = torch.empty(0)

        label = self.tensor_labels.iloc[idx, 4]
        image_path = self.tensor_labels.iloc[idx, 5]

        euclidian_masked = self.tensor_labels.iloc[idx, 6]
        euclidian_inverse_masked = self.tensor_labels.iloc[idx, 7]

        if self.transform:
            tensor = self.transform(tensor)
        if self.target_transform:
            label = self.target_transform(label)
        return image, masked, inverse_masked, label, image_path, euclidian_masked, euclidian_inverse_masked

In [ ]:
tensor_dataset = TensorDataset(gallery_tensor_dataset_path, "")
tensor_dataloader = DataLoader(tensor_dataset, batch_size=1, shuffle=False)

image_tensor_dict = {}
masked_tensor_dict = {}
label_dict = {}
image_path_dict = {}

for idx, data in enumerate(tqdm(tensor_dataloader)):
    image, masked, inverse_masked, label, image_path, euclidian_masked, euclidian_inverse_masked = data

    image_tensor_dict[idx] = image

    if masked.nelement() != 0 and inverse_masked.nelement() != 0:
      if euclidian_masked <= euclidian_inverse_masked:
        masked_tensor_dict[idx] = masked
      else:
        masked_tensor_dict[idx] = inverse_masked
    else:
      masked_tensor_dict[idx] = torch.empty(0)

    label_dict[idx] = label
    image_path_dict[idx] = image_path[0]

100%|██████████| 1067/1067 [00:02<00:00, 513.19it/s]


In [ ]:
print(image_tensor_dict[0])
print(masked_tensor_dict[0])
print(label_dict[0])
print(image_path_dict[0])

tensor([[ 0.9930, -0.6776, -0.1224,  ...,  1.1787,  0.1753, -0.7043]])
tensor([[ 1.0654, -0.6748, -0.1724,  ...,  1.2133,  0.1932, -0.7744]])
tensor([5019])
gallery/ambitious-tough-teal-from-asgard.jpg


In [ ]:
query_tensor_dataset = TensorDataset(query_tensor_dataset_path, "")
query_tensor_dataloader = DataLoader(query_tensor_dataset, batch_size=1, shuffle=False)

query_image_tensor_dict = {}
query_masked_tensor_dict = {}
query_label_dict = {}
query_image_path_dict = {}

for idx, data in enumerate(tqdm(query_tensor_dataloader)):
    image, masked, inverse_masked, label, image_path, euclidian_masked, euclidian_inverse_masked = data

    query_image_tensor_dict[idx] = image

    if masked.nelement() != 0 and inverse_masked.nelement() != 0:
      if euclidian_masked <= euclidian_inverse_masked:
        query_masked_tensor_dict[idx] = masked
      else:
        query_masked_tensor_dict[idx] = inverse_masked
    else:
      query_masked_tensor_dict[idx] = torch.empty(0)

    query_label_dict[idx] = label
    query_image_path_dict[idx] = image_path[0]

100%|██████████| 1935/1935 [00:02<00:00, 880.56it/s]


In [ ]:
print(query_image_tensor_dict[0])
print(query_masked_tensor_dict[0])
print(query_label_dict[0])
print(query_image_path_dict[0])

tensor([[ 0.6526,  0.3079, -0.2867,  ...,  0.5293,  0.1123, -0.2923]])
tensor([[ 0.7154,  0.1158, -0.3715,  ...,  0.3898,  0.2598, -0.3629]])
tensor([5013])
queries/magnetic-powerful-platypus-of-hail.jpeg


In [ ]:
MAX_VAL = 1000000

def AP(actual_label, predicted_labels):
    GPTs = predicted_labels.count(actual_label)
    if GPTs < 1:
        return 1
    AP = 0
    GPTs_found = 0
    for i in range(len(predicted_labels)):
        if predicted_labels[i] == actual_label:
            GPTs_found += 1
            AP += GPTs_found / (i + 1)
    return AP / GPTs

def mAP(APs):
    return np.average(APs)

def get_similarity_score(q, qm, g, gm):
    if gm.nelement() != 0 and qm.nelement() != 0:
      distance = torch.nn.functional.pairwise_distance(qm, gm, p=2)
    else:
      distance = torch.nn.functional.pairwise_distance(q, g, p=2)

    return torch.tensor(distance.item())


def get_mAP_of_model(query_tensor_dictionary, query_masked_tensor_dictionary, query_label_dictionary):
    APs_euclidean = []
    APs_manhattan = []
    APs_cosine = []
    for i in tqdm(range(len(query_tensor_dictionary))):
        query_tensor = query_tensor_dictionary[i].to('cuda')
        query_masked_tensor = query_masked_tensor_dictionary[i].to('cuda')

        euclidean = None
        pred_labels = None
        act_label = query_label_dictionary[i]

        for k in range(len(image_tensor_dict)):
          gallery_tensor = image_tensor_dict[k].to('cuda')
          gallery_masked_tensor = masked_tensor_dict[k].to('cuda')
          gallery_label = label_dict[k]

          if euclidean == None:
            euclidean = torch.unsqueeze(get_similarity_score(query_tensor, query_masked_tensor, gallery_tensor, gallery_masked_tensor), 0)
            pred_labels = gallery_label
          else:
            distance = torch.unsqueeze(get_similarity_score(query_tensor, query_masked_tensor, gallery_tensor, gallery_masked_tensor), 0)
            euclidean = torch.cat((euclidean, distance), dim=0)
            pred_labels = torch.cat((pred_labels, gallery_label), dim=0)

        euclidean_, euclidean_indices = torch.sort(euclidean)

        pred_labels_euclidean = pred_labels[euclidean_indices]

        APs_euclidean.append(AP(act_label.item(), pred_labels_euclidean.cpu().numpy().astype(int).tolist()))

    print('\nEuclidean distance mAP:', mAP(APs_euclidean))

In [2]:
get_mAP_of_model(query_image_tensor_dict, query_masked_tensor_dict, query_label_dict)

100%|██████████| 1935/1935 [06:41<00:00,  4.81it/s]
Euclidean distance mAP: 0.618921875807913


# Results

In [ ]:
# Grounding DINO & SAM, without cropping
# Euclidean distance mAP: 0.609121875807913

# Grounding DINO & SAM, with cropping
# Euclidean distance mAP: 0.59669641746546

# Grounding DINO, SAM & BLIP-2 without cropping
# Euclidean distance mAP: 0.61669641746546

# Grounding DINO, SAM & BLIP-2 with cropping
# Euclidean distance mAP: 0.618921875807913